<a href="https://colab.research.google.com/github/ctorney/abc-graph-nets/blob/main/testing_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#running with matched noise models, and new multiscale am sampler.


import numpy as np

import time
np.set_printoptions(suppress=True,precision=3)

import os, sys
from math import *
import tensorflow as tf 
import scipy 
import matplotlib.pyplot as plt


sys.path.append('..')


from gpabc import gp_abc
from gpabc import am_sampler
#from gabc import am_sampler_multi

#from sobol_seq import i4_sobol_generate
from scipy.stats import qmc
from scipy.stats import gaussian_kde

from simulations import zonal

#sobol_points = np.load('sobol_points_2d.npy')
    
##
sobol_listx = np.array([1.0,3.0,10.0,10.0]) #0,3,5,12])   
sobol_listy = np.array([9.0,12.0,10.0,5.0]) #15,15,13,9]) 

   


threadid = 1

num_reps = 1 #10
burnin = 100 #5000 #10000
mcmcsteps = 200 #1000 #8000
 
    

#for dl in range(di):
    
lali = sobol_listx[threadid]
latt = sobol_listy[threadid]
lrep = 1 # sobol_listlrep[threadid]
#    eta = sobol_listeta[threadid]
va = 1.5*pi #sobol_listva[threadid]
#        lali = sobol_points[dl,0]  #data instantiation
#        latt = sobol_points[dl,1] 
    
for data_rep in range(num_reps):             
    ## generate data
    L= 500 #200
    discard= 2000 #2500 #5000 
    N= 100 #500
    repeat =  2 #100 
    timesteps = 2 
    save_interval=1 #0 #1
    dt=0.1 #1 #0.1
    simulation_cls = zonal.zonal_model(N,timesteps=timesteps+discard,discard=discard,L=L,repeat=repeat, dt=dt,save_interval=save_interval)
        
    simulation_cls.run_sim(lrep, lali, latt, va)
        
    data_va=va
    data_latt=latt
    data_lali=lali
    data_lrep=lrep
        
    #DATA_y = [data_lali,data_latt,data_lrep,data_eta,data_vs,data_va,data_sigma]
        
        
    op, rot, nnd = simulation_cls.get_macro_states()
    avgOPDATA=np.zeros(repeat)
    avgROTDATA=np.zeros(repeat)
    avgNNDDATA=np.zeros(repeat)
        
        
    for i in range(repeat):
        avgOPDATA[i] = op[((i+1)*(timesteps-1))-1]  
        avgROTDATA[i] = rot[((i+1)*(timesteps-1))-1] 
        avgNNDDATA[i] = nnd[((i+1)*(timesteps-1))-1] 
        
    macrodata=  np.array([avgOPDATA,avgROTDATA,avgNNDDATA])

        
    def abc_likelihood_2d(sim_output,rc=None):
        theta_0 = sim_output

        ss_0 = macrodata
        theta_DATA0 = np.mean(ss_0,axis=-1) 
        sd0 =  np.std(ss_0,axis=-1) 
        cov = np.diag(sd0**2)
        repeat = sim_output.shape[1]
        
        k = sd0.shape[0]
        return np.log(1e-18 + 1/repeat * (((2*pi)**k)**0.5*np.product(sd0))*np.sum(scipy.stats.multivariate_normal(theta_DATA0,cov).pdf(theta_0.T)))


    def simulator_2d(params):
        repeat = 50    
        simulation_cls = zonal.zonal_model(N,timesteps+discard,discard=discard,repeat=repeat,L=L,dt=dt, save_interval=1,disable_progress=True) 

        #simulation_cls.run_sim(eta, params[2], params[1],params[0], vs, params[3], sigma) 
        simulation_cls.run_sim(lrep, params[0], params[1], va) #params[3])
        
        output = np.array(simulation_cls.get_macro_states()) 
        
        return output #np.squeeze([output[np.r_[0:2,3],None]]) 
                        
    #2D inference of l_ali and eta: 
    ndim = 2
    p_start = np.array([0.0,0.0])
    p_range = np.array([5.0,25.0]) 
   


    # use values for plotting the predicted GP
    X = np.array([np.linspace(p_start[0],p_start[0]+p_range[0],100),np.linspace(p_start[1],p_start[1]+p_range[1],100)])
    y_previous = np.full((100,100),np.log(1e-18))

            
    # number of waves
    n_wave = 2 #3 #10 
    n_points = 20 
    T = 0.0000001 #3
            

    abcGP = gp_abc.abcGP(p_start,p_range,ndim,n_points,T,simulator_2d,abc_likelihood_2d) 
            
            
    for i in range(n_wave):
        abcGP.runWave()                      
        #abcGP.update_rc()
  
        #plt.figure(figsize=(4,3))
        #plt.plot(abcGP.sobel_points[:,0],abcGP.sobel_points[:,1],'o',c='b',ms=4) 
        print("all x: ", abcGP.sobol_points[:,0])
        print("all y: ", abcGP.sobol_points[:,1])
        print(np.shape(abcGP.sobol_points[:,1]))
        
        abcGP.remove_implausible()  
        #plt.plot(abcGP.sobel_points[:,0],abcGP.sobel_points[:,1],'o',c='r',ms=4)
        print("plausible x: ", abcGP.sobol_points[:,0])
        print("plausible y: ", abcGP.sobol_points[:,1])
        print(np.shape(abcGP.sobol_points[:,1]))
        
    
        #plt.plot(data_lali,data_latt,'X',c='k',markersize=10) 
        # plot GP output
        #xx, yy = np.meshgrid(X[0], X[1], sparse=False)
        #x_grid = np.array([xx.flatten(), yy.flatten()])
        #if i>0:
        #    y_pred, y_std  = abcGP.predict_final(x_grid,remove_implausible=True)
        #else:
        #    y_pred, y_std  = abcGP.predict_final(x_grid,remove_implausible=False)
        #y_pred = y_pred.reshape(100,100)
        # for plotting we'll carry through the implausible regions from previous waves
        #y_pred[np.isinf(y_pred)]=y_previous[np.isinf(y_pred)]
        #y_previous = y_pred.copy()
        #h = plt.contourf(X[0], X[1], y_pred,50,cmap='viridis',vmax=min(0,np.max(y_pred)))  #0.1*
        #ax = plt.gca()
        #ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
        #plt.xlabel('$\eta$')
        #plt.ylabel('$l_{ali}$')
        #plt.title('Wave {}'.format(i+1))
        #plt.colorbar()
        #plt.savefig('zonal_macroscale_wave_LReverywave_2D' + str(i) + '.png',dpi=300)
    
        #plt.show()

        #np.save('y_pred_clr.npy',y_pred)  
            

          


100%|██████████| 2002/2002 [00:06<00:00, 302.64it/s]
20it [10:11, 30.57s/it]
reconstraining parameters GP_regression.rbf.lengthscale
reconstraining parameters GP_regression.rbf.lengthscale
0it [00:00, ?it/s]

all x:  [2.5   3.75  1.25  1.875 4.375 3.125 0.625 0.938 3.438 4.688 2.188 1.562
 4.062 2.812 0.312 0.469 2.969 4.219 1.719 2.344 4.844 3.594 1.094 0.781
 3.281 4.531 2.031 1.406 3.906 2.656 0.156 0.234 2.734 3.984 1.484 2.109
 4.609 3.359 0.859 1.172]
all y:  [12.5    6.25  18.75   9.375 21.875  3.125 15.625  7.812 20.312  1.562
 14.062  4.688 17.188 10.938 23.438 11.719 24.219  5.469 17.969  2.344
 14.844  8.594 21.094  3.906 16.406 10.156 22.656  7.031 19.531  0.781
 13.281  6.641 19.141  0.391 12.891  3.516 16.016  9.766 22.266  1.953]
(40,)
plausible x:  [2.5   3.75  1.25  1.875 4.375 3.125 0.625 0.938 3.438 4.688 2.188 1.562
 4.062 2.812 0.312 0.469 2.969 4.219 1.719 2.344 4.844 3.594 1.094 0.781
 3.281 4.531 2.031 1.406 3.906 2.656 0.156 0.234 2.734 3.984 1.484 2.109
 4.609 3.359 0.859 1.172]
plausible y:  [12.5    6.25  18.75   9.375 21.875  3.125 15.625  7.812 20.312  1.562
 14.062  4.688 17.188 10.938 23.438 11.719 24.219  5.469 17.969  2.344
 14.844  8.594 21.094  3.906 16.4

40it [10:11, 15.29s/it]
reconstraining parameters GP_regression.rbf.lengthscale
reconstraining parameters GP_regression.rbf.lengthscale


all x:  [2.5   3.75  1.25  1.875 4.375 3.125 0.625 0.938 3.438 4.688 2.188 1.562
 4.062 2.812 0.312 0.469 2.969 4.219 1.719 2.344 4.844 3.594 1.094 0.781
 3.281 4.531 2.031 1.406 3.906 2.656 0.156 0.234 2.734 3.984 1.484 2.109
 4.609 3.359 0.859 1.172 3.672 4.922 2.422 1.797 4.297 3.047 0.547 0.391
 2.891 4.141 1.641 2.266 4.766 3.516 1.016 0.703 3.203 4.453 1.953 1.328]
all y:  [12.5    6.25  18.75   9.375 21.875  3.125 15.625  7.812 20.312  1.562
 14.062  4.688 17.188 10.938 23.438 11.719 24.219  5.469 17.969  2.344
 14.844  8.594 21.094  3.906 16.406 10.156 22.656  7.031 19.531  0.781
 13.281  6.641 19.141  0.391 12.891  3.516 16.016  9.766 22.266  1.953
 14.453  8.203 20.703 11.328 23.828  5.078 17.578  5.859 18.359 12.109
 24.609  8.984 21.484  2.734 15.234 10.547 23.047  4.297 16.797  1.172]
(60,)
plausible x:  [2.5   3.75  1.25  1.875 4.375 3.125 0.625 0.938 3.438 4.688 2.188 1.562
 4.062 2.812 0.312 0.469 2.969 4.219 1.719 2.344 4.844 3.594 1.094 0.781
 3.281 4.531 2.031 1.406 

In [ ]:
n_wave

1

In [ ]:
def am_sampler(likelihood_function, ndim, init_p, prior, sigma, n_samples, burn_in, m): 
    cov_0 = np.diag(sigma)#0.1/np.sqrt(ndim)
    #idty = np.eye(ndim)
    #cov_0 = sigma_0**2*idty
    #cov_opt = sigma**2*idty
    
    mean = np.zeros(ndim)
    M2 = np.zeros((ndim,ndim))
    # array for samples
    samples = np.zeros((n_samples,ndim))
        
    # store initial value
    #samples[0] = init_p.squeeze()

    # reshape and compute the current likelihood
    z = np.squeeze(init_p)#,(ndim))
    
    l_cur, std_cur  = likelihood_function(z)
    l_cur = np.squeeze(l_cur)
     
    
    # check is finite and we're starting in plausible region
    if not np.isfinite(l_cur):
        print('implausible starting location')
        return
        
    # total iterations for number of samples
    iters = (n_samples * m) + burn_in

    # create random numbers outside the for loop
    #innov = np.random.normal(loc=0, scale=sigma, size=(iters,ndim))
    u = np.random.rand(iters) 
    for i in tqdm(range(iters)): 
    #for i in range(iters): 
        # new location for z
        #cand = z + innov[i]
        cand = generate_AM_candidate(z,M2,i,cov_0)

    
        l_cand, std_cand  = likelihood_function(np.squeeze(cand))
        l_cand = np.squeeze(l_cand)  
        
        if ndim==1:
            if cand<prior[0]:
                l_cand = - np.inf
            elif cand>prior[1]:
                l_cand = - np.inf
        else:        
            for q in range(ndim):
                if cand[q]<prior[q,0]:
                    l_cand = - np.inf
                elif cand[q]>prior[q,1]:
                    l_cand = - np.inf                           
                                   
                             

        # check not moving to implausible location
        if np.isfinite(l_cand):
            # use sample from GP for update to account for uncertainty
            l_cand_s = np.random.normal(loc=l_cand, scale=std_cand)
            l_cur_s = np.random.normal(loc=l_cur, scale=std_cur)
            # Accept or reject candidate
            if np.exp(l_cand_s - l_cur_s) > u[i]:
                z = cand
                l_cur = l_cand
                std_cur = std_cand     

        mean, M2 = update_moments(mean, M2, z, i)

In [ ]:
from tqdm import tqdm

In [ ]:
sigma = steps
ndim = 2
n_samples = 5
init_p = np.array([-10,25]) #startval
likelihood_function = abcGP.predict_final

cov_0 = np.diag(sigma)#0.1/np.sqrt(ndim)
#idty = np.eye(ndim)
#cov_0 = sigma_0**2*idty
#cov_opt = sigma**2*idty

mean = np.zeros(ndim)
M2 = np.zeros((ndim,ndim))
# array for samples
samples = np.zeros((n_samples,ndim))

# store initial value
#samples[0] = init_p.squeeze()

# reshape and compute the current likelihood
z = np.squeeze(init_p)#,(ndim))

l_cur, std_cur  = likelihood_function(z)
l_cur = np.squeeze(l_cur)

print(l_cur)

# check is finite and we're starting in plausible region
#if not np.isfinite(l_cur):
#    print('implausible starting location')
#    return

# total iterations for number of samples
iters = (n_samples * m) + burn_in

# create random numbers outside the for loop
#innov = np.random.normal(loc=0, scale=sigma, size=(iters,ndim))
u = np.random.rand(iters) 
for i in tqdm(range(iters)): 
#for i in range(iters): 
    # new location for z
    #cand = z + innov[i]
    cand = generate_AM_candidate(z,M2,i,cov_0)


    l_cand, std_cand  = likelihood_function(np.squeeze(cand))
    l_cand = np.squeeze(l_cand)  

    if ndim==1:
        if cand<prior[0]:
            l_cand = - np.inf
        elif cand>prior[1]:
            l_cand = - np.inf
    else:        
        for q in range(ndim):
            if cand[q]<prior[q,0]:
                l_cand = - np.inf
            elif cand[q]>prior[q,1]:
                l_cand = - np.inf                           



    # check not moving to implausible location
    if np.isfinite(l_cand):
        # use sample from GP for update to account for uncertainty
        l_cand_s = np.random.normal(loc=l_cand, scale=std_cand)
        l_cur_s = np.random.normal(loc=l_cur, scale=std_cur)
        # Accept or reject candidate
        if np.exp(l_cand_s - l_cur_s) > u[i]:
            z = cand
            l_cur = l_cand
            std_cur = std_cand     

    mean, M2 = update_moments(mean, M2, z, i)

-41.4465232845515


NameError: name 'm' is not defined

In [ ]:
np.diag(steps)

array([[0.094, 0.   ],
       [0.   , 0.477]])

In [ ]:
l_cur

array(-41.447)

In [ ]:
am_sampler(abcGP.predict_final,2,startval,prior,steps,n_samples=3, burn_in=burnin, m=100)

  0%|          | 0/400 [00:00<?, ?it/s]


NameError: name 'generate_AM_candidate' is not defined

In [ ]:
steps

array([0.094, 0.477])

In [ ]:
#sampling:
# random plausible point to start with
startval = abcGP.sobol_points[np.random.choice(abcGP.sobol_points.shape[0])]
# step size is 1/50th of the plausible range
steps = np.ptp(abcGP.sobol_points,axis=0)/50
prior = np.array(((0.0,5.0),(0.0,25.0)))
samples = am_sampler.am_sampler(abcGP.predict_final,2,startval,prior, steps,n_samples=mcmcsteps, burn_in=burnin, m=100)

